Old code can be seen at:
https://www.youtube.com/watch?v=qA6U4nIK62E&ab_channel=AntonioLonga
New code can be found at: 
https://github.com/pyg-team/pytorch_geometric/blob/master/examples/autoencoder.py

GAEs can only be used in a transductive setting
VGAEs might also be usable in an inductive setting

In [2]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges

In [4]:
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
# dataset.data

In [44]:
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = None
data

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327])

In [45]:
transform = T.RandomLinkSplit(is_undirected = True)
train_data, val_data, test_data = transform(data)
data = train_test_split_edges(data)


c:\Users\aske9\anaconda3\lib\site-packages\torch_geometric\deprecation.py:22: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


['__call__', '__cat_dim__', '__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__inc__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_edge_attr_cls', '_edge_to_layout', '_edges_to_layout', '_get_edge_index', '_get_tensor', '_get_tensor_size', '_multi_get_tensor', '_put_edge_index', '_put_tensor', '_remove_edge_index', '_remove_tensor', '_store', '_tensor_attr_cls', '_to_type', 'apply', 'apply_', 'batch', 'clone', 'coalesce', 'contains_isolated_nodes', 'contains_self_loops', 'contiguous', 'coo', 'cpu', 'csc', 'csr', 'cuda', 'debug', 'detach', 'detach_', 'edge_attr', 'edge_attrs', 'edg

In [54]:
# train_pos_edge_index are edges that are present in the graph
# While train_neg_edge_index are edges that are not present in the graph
print(train_data.train_pos_edge_index)
print(data.train_pos_edge_index)

AttributeError: 'GlobalStorage' object has no attribute 'train_pos_edge_index'

In [48]:
print(dir(data.train_pos_edge_index))

['H', 'T', '__abs__', '__add__', '__and__', '__array__', '__array_priority__', '__array_wrap__', '__bool__', '__class__', '__complex__', '__contains__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__div__', '__dlpack__', '__dlpack_device__', '__doc__', '__eq__', '__float__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__iand__', '__idiv__', '__ifloordiv__', '__ilshift__', '__imod__', '__imul__', '__index__', '__init__', '__init_subclass__', '__int__', '__invert__', '__ior__', '__ipow__', '__irshift__', '__isub__', '__iter__', '__itruediv__', '__ixor__', '__le__', '__len__', '__long__', '__lshift__', '__lt__', '__matmul__', '__mod__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__nonzero__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdiv__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rfloordiv__', '__rlshift__', '__rmatmul__', '__rmod__', '__rmul_

In [11]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        # Caches the normalization of the adjecency matrix "cached=True is only useful in case we have only one graph"
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True) # cached only for transductive learning

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

In [14]:
from torch_geometric.nn.models import GAE

In [49]:
# parameters
out_channels = 2
num_features = dataset.num_features
epochs = 100

# model
model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = train_data.x.to(device)
train_pos_edge_index = train_data.edge_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [50]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    #if args.variational:
    #   loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [9]:
for epoch in range(1, epochs + 1):
    loss = train()

    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

Epoch: 001, AUC: 0.6215, AP: 0.6563
Epoch: 002, AUC: 0.6420, AP: 0.6774
Epoch: 003, AUC: 0.6471, AP: 0.6826
Epoch: 004, AUC: 0.6496, AP: 0.6857
Epoch: 005, AUC: 0.6519, AP: 0.6884
Epoch: 006, AUC: 0.6539, AP: 0.6905
Epoch: 007, AUC: 0.6553, AP: 0.6920
Epoch: 008, AUC: 0.6568, AP: 0.6936
Epoch: 009, AUC: 0.6585, AP: 0.6949
Epoch: 010, AUC: 0.6600, AP: 0.6964
Epoch: 011, AUC: 0.6609, AP: 0.6976
Epoch: 012, AUC: 0.6621, AP: 0.6990
Epoch: 013, AUC: 0.6627, AP: 0.7003
Epoch: 014, AUC: 0.6637, AP: 0.7022
Epoch: 015, AUC: 0.6641, AP: 0.7038
Epoch: 016, AUC: 0.6642, AP: 0.7056
Epoch: 017, AUC: 0.6639, AP: 0.7073
Epoch: 018, AUC: 0.6634, AP: 0.7090
Epoch: 019, AUC: 0.6628, AP: 0.7106
Epoch: 020, AUC: 0.6620, AP: 0.7119
Epoch: 021, AUC: 0.6615, AP: 0.7134
Epoch: 022, AUC: 0.6611, AP: 0.7144
Epoch: 023, AUC: 0.6602, AP: 0.7153
Epoch: 024, AUC: 0.6595, AP: 0.7160
Epoch: 025, AUC: 0.6590, AP: 0.7166
Epoch: 026, AUC: 0.6582, AP: 0.7165
Epoch: 027, AUC: 0.6576, AP: 0.7166
Epoch: 028, AUC: 0.6569, AP: